In [1]:
import pandas as pd
from datetime import timedelta
import json
import os
import glob
from pprint import pprint

In [13]:
# read the data from the json files
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
target_dir = os.path.join(parent_dir,'stock_scrape_load','data')
file_path = f"{target_dir}/*.json"

# declare empty list to store data
data = []

# Loop through all json files and update data list
for file in glob.glob(file_path):
    with open(file, 'r') as json_file:
        json_data = json.load(json_file)
        data.append(json_data)

# Create Dataframe
prices_df = pd.concat([pd.DataFrame(d) for d in data], ignore_index = True)
prices_df['date'] = pd.to_datetime(prices_df['date'])

# View that sweet sweet extended dataframe!
prices_df = prices_df.sort_values(['ticker','date'])
prices_df.head()


,ticker,date,sector,52WeekChange,ask,bid,52WeekHigh,52WeekLow,dailyOpen,previousClose,dailyVolume,quickRatio,10DayAverageVolume,marketCap,dayHigh,dayLow
0,AAPL,2023-07-11,Technology,29.273487,188.24,188.12,194.48,124.17,189.16,188.61,46509718.0,0.764,51012280.0,2.958254e+12,NaN,NaN
100,AAPL,2023-07-12,Technology,29.273487,0.00,0.00,194.48,124.17,189.68,188.08,26845497.0,0.764,51012280.0,2.970052e+12,NaN,NaN
200,AAPL,2023-07-13,Technology,27.817070,190.33,190.32,194.48,124.17,190.50,189.77,38321242.0,0.764,52013490.0,2.996947e+12,NaN,NaN
300,AAPL,2023-07-14,Technology,26.882862,190.01,190.00,194.48,124.17,190.23,190.54,21271537.0,0.764,51027070.0,2.984993e+12,191.175,189.725
24,ABBV,2023-07-11,Healthcare,-10.897141,135.73,135.00,168.11,130.96,134.00,134.49,5104083.0,0.659,6124920.0,2.391848e+11,NaN,NaN


In [14]:

def get_closing_date(row):
    reported_day = row['date']
    actual_close_day = row['datePrevClose']

    if reported_day.weekday() == 0: # When the previous day is Monday
        actual_close_day -= timedelta(days=2) # now it is Friday

    if pd.isnull(actual_close_day): # Check if actual_close_day is NaT
        actual_close_day = row['date'] - timedelta(days=1) # Fill with yesterday's date

    return actual_close_day # provide the date of the most recent Friday

prices_df['datePrevClose'] = prices_df['date'].shift(1) # Create new column, and locate it next to the current prices date
prices_df['datePrevClose'] = prices_df.apply(get_closing_date, axis=1)
prices_df['dailyClose'] = prices_df['previousClose'].shift(-1)

prices_df.head(18)


,ticker,date,sector,52WeekChange,ask,bid,52WeekHigh,52WeekLow,dailyOpen,previousClose,dailyVolume,quickRatio,10DayAverageVolume,marketCap,dayHigh,dayLow,datePrevClose,dailyClose
0,AAPL,2023-07-11,Technology,29.273487,188.24,188.12,194.48,124.17,189.160,188.61,46509718.0,0.764,51012280.0,2.958254e+12,NaN,NaN,2023-07-10,188.08
100,AAPL,2023-07-12,Technology,29.273487,0.00,0.00,194.48,124.17,189.680,188.08,26845497.0,0.764,51012280.0,2.970052e+12,NaN,NaN,2023-07-11,189.77
200,AAPL,2023-07-13,Technology,27.817070,190.33,190.32,194.48,124.17,190.500,189.77,38321242.0,0.764,52013490.0,2.996947e+12,NaN,NaN,2023-07-12,190.54
300,AAPL,2023-07-14,Technology,26.882862,190.01,190.00,194.48,124.17,190.230,190.54,21271537.0,0.764,51027070.0,2.984993e+12,191.175,189.725,2023-07-13,134.49
24,ABBV,2023-07-11,Healthcare,-10.897141,135.73,135.00,168.11,130.96,134.000,134.49,5104083.0,0.659,6124920.0,2.391848e+11,NaN,NaN,2023-07-14,135.57
124,ABBV,2023-07-12,Healthcare,-10.897141,136.54,136.47,168.11,130.96,135.820,135.57,1097186.0,0.659,6124920.0,2.399611e+11,NaN,NaN,2023-07-11,133.50
224,ABBV,2023-07-13,Healthcare,-10.276520,133.76,133.59,168.11,130.96,132.930,133.50,4607808.0,0.659,5835300.0,2.356915e+11,NaN,NaN,2023-07-12,133.59
324,ABBV,2023-07-14,Healthcare,-13.038665,135.81,135.80,168.11,130.96,134.730,133.59,1968010.0,0.659,5607370.0,2.425563e+11,136.065,134.120,2023-07-13,131.71
93,ABNB,2023-07-11,Consumer Cyclical,43.810116,137.85,137.65,144.63,81.91,129.450,131.71,10028640.0,0.883,4888010.0,8.668526e+10,NaN,NaN,2023-07-14,137.54
193,ABNB,2023-07-12,Consumer Cyclical,43.810116,0.00,0.00,144.63,81.91,139.350,137.54,2845669.0,0.883,4888010.0,8.633227e+10,NaN,NaN,2023-07-11,137.02


In [24]:
# Pick out the columns I want included in final json
ticker = pd.DataFrame(prices_df[['ticker','date','datePrevClose','dailyOpen','dayHigh', 'dayLow','dailyClose','previousClose','dailyVolume','ask','bid']])

# json cannot work with datetime dtypes
ticker[['date','datePrevClose']] = ticker[['date','datePrevClose']].astype(str)
print(f"There are {len(prices_df.value_counts(['date']))} days of data represented")
ticker.head(18)


There are 4 days of data represented


,ticker,date,datePrevClose,dailyOpen,dayHigh,dayLow,dailyClose,previousClose,dailyVolume,ask,bid
0,AAPL,2023-07-11,2023-07-10,189.160,NaN,NaN,188.08,188.61,46509718.0,188.24,188.12
100,AAPL,2023-07-12,2023-07-11,189.680,NaN,NaN,189.77,188.08,26845497.0,0.00,0.00
200,AAPL,2023-07-13,2023-07-12,190.500,NaN,NaN,190.54,189.77,38321242.0,190.33,190.32
300,AAPL,2023-07-14,2023-07-13,190.230,191.175,189.725,134.49,190.54,21271537.0,190.01,190.00
24,ABBV,2023-07-11,2023-07-14,134.000,NaN,NaN,135.57,134.49,5104083.0,135.73,135.00
124,ABBV,2023-07-12,2023-07-11,135.820,NaN,NaN,133.50,135.57,1097186.0,136.54,136.47
224,ABBV,2023-07-13,2023-07-12,132.930,NaN,NaN,133.59,133.50,4607808.0,133.76,133.59
324,ABBV,2023-07-14,2023-07-13,134.730,136.065,134.120,131.71,133.59,1968010.0,135.81,135.80
93,ABNB,2023-07-11,2023-07-14,129.450,NaN,NaN,137.54,131.71,10028640.0,137.85,137.65
193,ABNB,2023-07-12,2023-07-11,139.350,NaN,NaN,137.02,137.54,2845669.0,0.00,0.00


In [26]:
historical_json = []
grouped_tickers = prices_df.groupby(['ticker'])
grouped_dates = prices_df.groupby(['date'])
data = {
    "tickers": list(grouped_tickers.groups.keys()),
    "dates": list(grouped_dates.groups.keys()),
    "history": []
}

for ticker, group in grouped_tickers:
    ticker_data = {
        "ticker": ticker,
        "stats": []
    }
    
    for date, day_group in group.groupby('date'):
        date_data ={
            # "date": date,
            "market" : []
        }
    
        for index, row in day_group.iterrows():
            entry_data = {
                # "symbol": row['ticker'],
                "date": row['date'],
                "datePrevClose": row['datePrevClose'],
                "dailyOpen": row['dailyOpen'],
                "dayHigh": row['dayHigh'],
                "dayLow": row['dayLow'],
                # "dailyClose": row['dailyClose'],
                "previousClose": row['previousClose'],
                "ask": row['ask'],
                "bid": row['bid'],
                "dailyVolume": row['dailyVolume']
            }
            date_data["market"].append(entry_data)
        ticker_data["stats"].append(date_data)
    data["history"].append(ticker_data)
    #     ticker_data["day"].append(day_data)
    # data["days"].append(ticker_data)

json_data = json.dumps(data, indent=4)
with open("historical.json", "w") as json_file:
    json_file.write(json_data)
    

TypeError: Object of type Timestamp is not JSON serializable

In [9]:
## Old code-- delete later

# prices_df.to_json(
#     path_or_buf = 'all_prices_xdf.json',
#     orient = 'records',
#     date_format = 'iso')

# Old code - second attempt - delete later    json_file.write(json_data)



# Old code - delete later

#  THe follow is an attempt to reorganize the datafram into a structured json.  Then I realized it did not really matter
# json_data = {}

# for index, row in prices_df.iterrows():
#     date = row['date']
#     open = row['dailyOpen'], 
#     close = row['previousClose'], 
#     volume = row['dailyVolume'],
#     ticker = row['ticker'], 
#     cap = row['marketCap'], 
#     quick = row['quickRatio'],
#     sector = row['sector'],
#     low = row['52WeekLow'], 
#     high = row['52WeekHigh'], 
#     change = row['52WeekChange'],
#     ave_vol = row['10DayAverageVolume']

#     if date not in json_data:
#         json_data[date] = {'price':[],'info':[],'stats':[]}

#     json_data[date]['price'].append({'ticker':ticker,
#                                      'dailyOpen':open,
#                                      'previousClose':close,
#                                      'dailyVolume':volume
#                                                       })
#     json_data[date]['info'].append({'sector':sector,
#                                     'marketCap':cap,
#                                     })
#     json_data[date]['stats'].append({'52WeekLow':low,
#                                      '52WeekHigh':high, 
#                                      '52WeekChange': change,
#                                      '10DayAverageVolume':ave_vol
#                                      })
# # price_json = json.dumps(json_data)

# # pprint(price_json)

# prices = pd.DataFrame(json_data)
# prices.to_json('all_prices.json')


# Old code --  delet later

# Same commetn as previous cell

# output_path = f"{target_dir}/all_prices.json"

# with open('all_prices.json','w') as file:
#     file.write(price_json)